In [1]:
import os

In [2]:
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2")

In [3]:
%pwd

'/Users/richcriticism/Documents/Documents/Bhavya_code/machine learning/CorrectPose2'

In [4]:
import tensorflow as tf
import keras


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    all_params: dict
    params_batch_size: int
    eval_data_path: Path
    eval_data_target: Path


In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/base_model.h5",
            all_params=self.params,
            params_batch_size=self.params.EVAL_BATCH_SIZE,
            eval_data_path = "artifacts/data_extraction/extracted_data/eval_Data_X.csv",
            eval_data_target = "artifacts/data_extraction/extracted_data/eval_Data_Y.csv"
        )
        return eval_config


In [8]:
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np

In [9]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config


    def extract_eval_data(self, eval_data_x : Path, eval_data_y : Path):
        X = pd.read_csv(eval_data_x)
        Y = pd.read_csv(eval_data_y)
        X = X.drop(['Unnamed: 0'], axis = 1)
        return X,Y


    def _eval_data_generator(self, X, Y, params_batch_size):

        num_samples = len(X)

        iterations = 0
        max_iterations = num_samples//params_batch_size
        while iterations < max_iterations:

            indices = np.random.permutation(num_samples)
            for i in range(0, num_samples):
                print(i)
                if i+params_batch_size > num_samples:
                    break
                batch_indices = indices[i:i+params_batch_size]
                batch_x = X.iloc[batch_indices]
                batch_y = Y.iloc[batch_indices]
                yield np.array(batch_x), np.array(batch_y)

            iterations = iterations + 1
        
    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01), loss = tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
        X, Y = self.extract_eval_data(self.config.eval_data_path, self.config.eval_data_target)
        eval_data_generator = self._eval_data_generator(X,Y, self.config.params_batch_size)
        self.score = self.model.evaluate(eval_data_generator)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy": self.score[1]}
        save_json(path = Path("scores.json"), data = scores)

In [10]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-04-11 19:10:22,595: INFO : common : yaml file : config/config.yaml loaded successfully]
[2024-04-11 19:10:22,597: INFO : common : yaml file : params.yaml loaded successfully]
[2024-04-11 19:10:22,598: INFO : common : creating directory at path: artifacts]
[2024-04-11 19:10:22,701: WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
0
1
      1/Unknown 0s 193ms/step - accuracy: 1.0000 - loss: 0.0000e+002
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
     73/Unknown 0s 695us/step - accuracy: 1.0000 - loss: 0.0000e+0023
24
25
26
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 1.0000 - loss: 0.0000e+00
[2024-04-11 19:10:23,017: INFO : com

2024-04-11 19:10:23.003857: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
